# 2D Fourier Transform

In [1]:
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import numpy as np
images = [ cv2.imread("./data/00{}.jpg".format(i), cv2.IMREAD_GRAYSCALE) for i in range(1, 7)] 
cv2.imshow("i", images[1])
cv2.waitKey(0)

48

## Matrix Multiplication

In [6]:
class Matrix:
    def __init__(self, data):
        self.data = data
    
    def __repr__(self):
        return str(self.data)
    
    def __len__(self):
        return self.shape[0]
    
    def __getitem__(self, i):
        return self.data[i]
    
    @property
    def shape(self):
        return (len(self.data), len(self.data[0]))
    
    def matmul(self, B):
        result = [[0 for _ in range(B.shape[1])] for _ in range(self.shape[0])]
        for i in range(self.shape[0]):
            for j in range(B.shape[1]):
                sum = 0
                for k in range(self.shape[1]):
                    sum += self[i][k] * B[k][j]
                result[i][j] = sum 
        return result
    
    def transpose(self):
        result = [[0 for _ in range(self.shape[0])] for _ in range(self.shape[1])]
        for i in range(self.shape[0]):
            for j in range(self.shape[1]):
                result[j][i] = self[i][j]
                
    
    def exp(self):
        result = self.data.copy() 
        for i in range(self.shape[0]):
            for j in range(self.shape[1]):
                result[i][j] = self[i][j] * np.exp(-2j* np.pi )
        return Matrix(result)
                
    
A = Matrix([[1, 2, 3], [4, 5, 6]])
B = Matrix([[1, 2], [3, 4], [5, 6]])

In [8]:
def fourier(img):
    mat = Matrix(img)
    M, N = mat.shape 
    result = [[0 for _ in range(N)] for _ in range(M)]
    
    for m in range(M):
        for n in range(N):
            sum = 0
            for u in range(M):
                for v in range(N):
                    sum += mat[u][v] * np.exp(-2j * np.pi * (m * u / M + n * v / N))
            result[m][n] = sum 
    
    return Matrix(result)
def fourier_2d(img):
    mat = Matrix(img) 
    fourier_1d = fourier(mat) # along x axis 
    result = fourier(fourier_1d.transpose()).transpose() # along y axis 
    
    
    

In [14]:
a = [[3,2,1],[5,6,7], [1,2,3]]

fourier(a)


[[(30+0j), (-1.500000000000003+0.866025403784435j), (-1.499999999999996-0.8660254037844468j)], [(-6-10.392304845413268j), (3.0000000000000004-1.7320508075688794j), (2.999999999999993+1.7320508075688719j)], [(-6.0000000000000036+10.392304845413257j), (3.0000000000000018-1.732050807568882j), (2.999999999999999+1.7320508075688732j)]]

In [16]:
a = 1 + 2j

2.0